In [1]:
import anthropic
import os 
import config 

os.environ["ANTHROPIC_API_KEY"] = config.claude_api_key


In [41]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a AI critic who is evalauting a conversation between a patient and doctor. give 3 peices of feedback to improve the interaction",
    messages=[
        {
            "role": "user",
            "content": "Hi, I have a cold"      
        },
        {
            "role": "assistant",
            "content": "Hi, I am sorry to hear that, what are your symptoms"      
        },
        {
            "role": "user",
            "content": "I have a runny nose and a headache"      
        },
        {
            "role": "assistant",
            "content": "I would reccommend drinking some water"      
        },
        {
            "role": "user",
            "content": "that's good to know thank you, what else should i do?"      
        },

    ]
)

In [43]:
message

Message(id='msg_01MuACpF9jxtkGvGjX1iGQ4f', content=[TextBlock(text="Here are three pieces of feedback to improve the interaction between the patient and doctor:\n\n1. Gather more detailed information: The doctor should ask more specific questions about the patient's symptoms, such as how long they've been experiencing them, their severity, and if there are any other symptoms present. This would help in making a more accurate assessment and providing better advice.\n\n2. Provide a more comprehensive treatment plan: The doctor's recommendation to drink water is good, but it's quite limited. They should offer a more complete set of suggestions, such as rest, over-the-counter medications for symptom relief, and when to seek further medical attention if symptoms worsen or persist.\n\n3. Explain the rationale behind recommendations: The doctor should briefly explain why they're recommending certain actions. For example, they could explain that staying hydrated helps thin mucus and relieve co

In [48]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a doctor repsonding to a patient, you have been going through a conversation with a patient and have a just recieved feedbck from a critic, take in the feedback to make your next response to the patient",
    messages=[
        {
            "role": "user",
            "content": "Hi, I have a cold"      
        },
        {
            "role": "assistant",
            "content": "Hi, I am sorry to hear that, what are your symptoms"      
        },
        {
            "role": "user",
            "content": "I have a runny nose and a headache"      
        },
        {
            "role": "assistant",
            "content": "I would reccommend drinking some water"      
        },
        {
            "role": "user",
            "content": "Here are three pieces of feedback to improve the interaction between the patient and doctor:\n\n1. Gather more detailed information: The doctor should ask more specific questions about the patient's symptoms, such as how long they've been experiencing them, their severity, and if there are any other symptoms present. This would help in making a more accurate assessment and providing better advice.\n\n2. Provide a more comprehensive treatment plan: The doctor's recommendation to drink water is good, but it's quite limited. They should offer a more complete set of suggestions, such as rest, over-the-counter medications for symptom relief, and when to seek further medical attention if symptoms worsen or persist.\n\n3. Explain the rationale behind recommendations: The doctor should briefly explain why they're recommending certain actions. For example, they could explain that staying hydrated helps thin mucus and relieve congestion. This helps educate the patient and increases the likelihood that they'll follow the advice.\n\nBy implementing these improvements, the interaction would be more thorough, informative, and beneficial for the patient."      
        },
        {
            "role": "assistant",
            "content": "I have taken in this feedbakc and will improve the nmext round of conversation" 
        },
        {
            "role": "user",
            "content": "I have a cold" 
        },



    ]
)

In [49]:
message

Message(id='msg_01XXb8WcvDPdCopUunr3tK9n', content=[TextBlock(text="I'm sorry to hear that you're not feeling well. To better understand your situation and provide more comprehensive advice, I'd like to ask you a few questions:\n\n1. How long have you been experiencing cold symptoms?\n2. Besides the runny nose and headache you mentioned earlier, are you experiencing any other symptoms such as fever, sore throat, cough, or body aches?\n3. On a scale of 1 to 10, with 10 being the most severe, how would you rate your symptoms?\n\nBased on your answers, I can provide more tailored advice. In the meantime, here are some general recommendations to help manage your cold:\n\n1. Stay hydrated by drinking plenty of water, herbal tea, or clear broths. This helps thin mucus and relieve congestion.\n2. Get plenty of rest to allow your body to focus on fighting the infection.\n3. Consider over-the-counter medications like acetaminophen or ibuprofen for headache relief and to reduce any fever.\n4. Us

In [ ]:
class Agent:
    def __init__(self, name, role, system_prompt):
        self.name = name
        self.role = role
        self.system_prompt = system_prompt
        self.conversation_history = []

    def respond(self, message):
        response = f"{self.role} {self.name} response: {message}"
        self.conversation_history.append(response)
        return response

    def reset_conversation(self):
        self.conversation_history = []

class Doctor(Agent):
    def __init__(self, name, system_prompt):
        super().__init__(name, "Doctor", system_prompt)

    def respond(self, message, critic_feedback=None):
        context = "\n".join(self.conversation_history)
        if critic_feedback:
            context += f"\nCritic's feedback: {critic_feedback}"
        # In a real scenario, you would pass the context and critic_feedback to the LLM API
        response = f"{self.role} {self.name} response (considering previous context and feedback): {message}"
        self.conversation_history.append(response)
        return response

class Patient(Agent):
    def __init__(self, name, vignette, system_prompt):
        super().__init__(name, "Patient", system_prompt)
        self.vignette = vignette

    def respond(self, message):
        context = "\n".join(self.conversation_history)
        # In a real scenario, you would pass the context to the LLM API
        response = f"{self.role} {self.name} ({self.vignette}) response (considering previous context): {message}"
        self.conversation_history.append(response)
        return response

class Critic(Agent):
    def __init__(self, name, system_prompt):
        super().__init__(name, "Critic", system_prompt)

    def critique(self, conversation):
        # In a real scenario, you would pass the conversation to the LLM API
        return f"Critic {self.name} advice: Based on the conversation, the doctor should improve by focusing more on patient concerns and asking more specific questions."

class Moderator(Agent):
    def __init__(self, name, system_prompt):
        super().__init__(name, "Moderator", system_prompt)

    def is_conversation_finished(self, conversation):
        return len(conversation) >= 5 or "goodbye" in conversation[-1].lower()

def simulate_conversation(doctor, patient, critic, moderator, max_turns=10, max_rounds=3):
    for round in range(max_rounds):
        print(f"\n--- Round {round + 1} ---")
        conversation = []
        
        for _ in range(max_turns):
            if moderator.is_conversation_finished(conversation):
                break
            
            if len(conversation) % 2 == 0:
                message = doctor.respond("Hello, How can I help you today?")
            else:
                message = patient.respond("I'm not feeling well.")
            conversation.append(message)
            print(message)

        print("\nConversation round finished.")
        critic_feedback = critic.critique(conversation)
        print(critic_feedback)

        if round < max_rounds - 1:
            print("\nDoctor considering feedback and restarting conversation...")
            doctor.respond("Hello, How can I help you today?", critic_feedback)
            patient.reset_conversation()
    
    return conversation

# Default system prompts
default_doctor_prompt = "You are a compassionate doctor. Diagnose and treat patients effectively."
default_patient_prompt = "You are a patient describing your symptoms. Be clear but not too technical."
default_critic_prompt = "You are a medical expert. Provide constructive feedback to improve doctor-patient interactions."
default_moderator_prompt = "You are a conversation moderator. Determine when the doctor-patient conversation is complete."

def create_agents(doctor_name, patient_name, patient_vignette, critic_name, moderator_name,
                  doctor_prompt=default_doctor_prompt,
                  patient_prompt=default_patient_prompt,
                  critic_prompt=default_critic_prompt,
                  moderator_prompt=default_moderator_prompt):
    doctor = Doctor(doctor_name, doctor_prompt)
    patient = Patient(patient_name, patient_vignette, patient_prompt)
    critic = Critic(critic_name, critic_prompt)
    moderator = Moderator(moderator_name, moderator_prompt)
    return doctor, patient, critic, moderator

# Example usage
doctor, patient, critic, moderator = create_agents(
    "Dr. Smith", 
    "John Doe", 
    "35-year-old male with persistent cough",
    "Dr. Johnson",
    "Mod"
)

# Run simulation
final_conversation = simulate_conversation(doctor, patient, critic, moderator)

In [ ]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)

In [8]:
import anthropic
import time

# Initialize the Anthropic client
client = anthropic.Anthropic()

def agent_conversation(agent1_name, agent2_name, initial_prompt, num_turns=3):
    conversation = initial_prompt
    
    for _ in range(num_turns):
        # Agent 1's turn
        response1 = client.messages.create(
            prompt=f"{conversation}\n\n{agent1_name}:",
            model="claude-3-opus-20240229",
            max_tokens=300,
            system="You are an empathetic doctor",
            stop_sequences=[f"\n{agent2_name}:"]
        )
        conversation += f"\n\n{agent1_name}: {response1.messages}"
        print(f"{agent1_name}: {response1.messages}")
        
        time.sleep(1)  # Add a small delay between requests
        
        # Agent 2's turn
        response2 = client.completion(
            prompt=f"{conversation}\n\n{agent2_name}:",
            model="claude-3-opus-20240229",
            max_tokens=300,
            system="You are a patient with a cold",
            stop_sequences=[f"\n{agent1_name}:"]
        )
        conversation += f"\n\n{agent2_name}: {response2.messages}"
        print(f"{agent2_name}: {response2.messages}")
        
        time.sleep(1)  # Add a small delay between requests

    return conversation

# Example usage
initial_prompt = """
Doctor: How can I help you today? """

full_conversation = agent_conversation("Doctor", "Patient", initial_prompt)
print("\nFull conversation:")
print(full_conversation)

TypeError: Missing required arguments; Expected either ('max_tokens', 'messages' and 'model') or ('max_tokens', 'messages', 'model' and 'stream') arguments to be given

In [ ]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]

In [ ]:
critic_instructions = "You are an AI critic providing feedback to a doctor after they have completed a conversation with a patient. \
All of the previous conversation is the completed conversation. \
Provide feedback based on the following dimensions: Fostering the relationship, Gathering inofrmation, \
Providing information, making decisions,\
Enabling Disease and Treatment-Related Behavior, Responding to Emotions.\
Your feedback will only be seen by the doctor. Give you feedback in 50 words or less"

def generate_response_critic(messages):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        temperature=1,
        system= critic_instructions,
        max_tokens=100,
        messages=[]
    )
    return {"role": "assistant", "content": response.content[0].text}, {"role": "user", "content": response.content[0].text}

In [127]:
import anthropic

# Initialize OpenAI client
client = anthropic.Anthropic()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot.\
Give your responses in 60 words or less"

doctor_instructions = "You are an clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.Do not reveal you are an AI bot. Give your repsonses in 70 words or less"

critic_instructions = "You are an AI critic providing feedback to a doctor after they have completed a conversation with a patient. \
All of the previous conversation is the completed conversation. \
Provide feedback based on the following dimensions: Fostering the relationship, Gathering inofrmation, \
Providing information, making decisions,\
Enabling Disease and Treatment-Related Behavior, Responding to Emotions.\
Your feedback will only be seen by the doctor. Give you feedback in 100 words or less"

# Function to generate response from a given agent
def generate_response_doctor(messages):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        temperature=1,
        system= doctor_instructions,
        max_tokens=1000,
        messages=messages
    )
    return {"role": "assistant", "content": response.content[0].text}, {"role": "user", "content": response.content[0].text}


def generate_response_patient(messages):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        temperature=1,
        system= patient_instructions,
        max_tokens=1000,
        messages=messages
    )
    return {"role": "assistant", "content": response.content[0].text}, {"role": "user", "content": response.content[0].text}

def generate_response_critic(messages):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        temperature=1,
        system= critic_instructions,
        max_tokens=1000,
        messages=messages
    )
    return {"role": "assistant", "content": response.content[0].text}, {"role": "user", "content": response.content[0].text}


def conversation_to_string(conversation):
    readable_string = ""
    for message in conversation:
        role = "Doctor" if message['role'] == 'user' else "Patient"
        content = message['content']
        readable_string += f"{role}: {content}\n"
    return readable_string.strip()



# Initialize agents' first messages
conversation_context_doctor = []
conversation_context_patient = [{"role": "user", "content": "Hello, how are you?"}]
conversation_context_critic = []


# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower() or "have a great day" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, conversation_context_critic, max_turns=10):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient 
        patient_response = generate_response_patient(patient_context)
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor
        doctors_response = generate_response_doctor(doctor_context)
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")


    
    doctor_patient_dialogue = conversation_to_string(conversation_context_patient)
    critic_context =  [{"role": "user", "content": "Here is the complete conversation between the Doctor and Patient.\n"+ doctor_patient_dialogue+ "\n Give feedback to the doctor "}]
    critic_feedback = generate_response_critic(critic_context)
    conversation_context_critic.append(critic_feedback[0])
    critic_feedback_to_doctor = "Here is some feedback on your previous interaction with the patient\n" + critic_feedback[1]["content"] + "\nThe conversation with the patient will start again. Incorperate the feedback given into your responses"
    conversation_context_doctor.extend([
        {"role": "user", "content": critic_feedback_to_doctor},
        {"role": "assistant", "content": "I understand and have acknowleged the feedback. I will incoperate it into the following conversation"},
        {"role": "user", "content": "Now introduce yourself to the patient"},
        {"role": "assistant", "content": "Hello, how are you?"},
        ])

    # Print Critic's feedback
    print(f"Critic: {critic_feedback[0]['content']}")

    conversation_context_patient =[{"role": "user", "content": "Hello, how are you?"}] 

    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient 
        patient_response = generate_response_patient(patient_context)
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor
        doctors_response = generate_response_doctor(doctor_context)
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")


    return conversation_context_doctor, conversation_context_patient

# Run the chat simulation
conversation_context_doctor, conversation_context_patient = chat_between_agents(
    conversation_context_doctor, conversation_context_patient, conversation_context_critic
)

Patient: Hello doctor. I'm doing okay, but I've been having some issues with my right hand that are concerning me. I'm experiencing numbness and tingling in my fingers, as well as pain in my hand and wrist. It's been going on for a few months now and is starting to affect my daily life.
Doctor: I'm sorry to hear you're experiencing these symptoms. Can you tell me more about when the numbness and tingling occur? For example, is it constant or does it come and go? Also, have you noticed any weakness in your hand or difficulty gripping objects? This information will help me better understand your condition.
Patient: The numbness and tingling come and go, but they're worst at night. It often wakes me up. I haven't noticed any weakness, but I do have trouble gripping objects sometimes due to the pain. It's especially bad when I'm typing or using a mouse at work. Should I be worried about weakness too?
Doctor: Thank you for providing more details. The symptoms you describe, especially worsen

In [60]:
def conversation_to_string(conversation):
    readable_string = ""
    for message in conversation:
        role = "Doctor" if message['role'] == 'user' else "Patient"
        content = message['content']
        readable_string += f"{role}: {content}\n"
    return readable_string.strip()

# Example usage
conversation = [
    {'role': 'user', 'content': 'Hello, how are you?'},
    {'role': 'assistant', 'content': "Hello doctor. I'm doing okay, but I"},
    {'role': 'user', 'content': "Hello there. I'm glad to hear you're"},
    {'role': 'assistant', 'content': "I'm sorry, I didn't mean to interrupt"},
    {'role': 'user', 'content': 'No need to apologize. Please, go ahead'}
]

converted_conversation = conversation_to_string(conversation)


In [61]:
converted_conversation

"Doctor: Hello, how are you?\nPatient: Hello doctor. I'm doing okay, but I\nDoctor: Hello there. I'm glad to hear you're\nPatient: I'm sorry, I didn't mean to interrupt\nDoctor: No need to apologize. Please, go ahead"

In [66]:
critic_context =  [{"role": "user", "content": "Here is the complete conversation between the Doctor and Patient.\n"+ converted_conversation+ "\n Give feedback to the doctor "}]

In [65]:
critic_context

[{'role': 'user',
  'content': "Here is the conversation between the Doctor and Patient.\nDoctor: Hello, how are you?\nPatient: Hello doctor. I'm doing okay, but I\nDoctor: Hello there. I'm glad to hear you're\nPatient: I'm sorry, I didn't mean to interrupt\nDoctor: No need to apologize. Please, go ahead\n Give feedback to the doctor "}]

In [105]:
conversation_context_doctor = []
conversation_context_critic = []

In [111]:
def generate_response_critic(messages):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        temperature=1,
        system= critic_instructions,
        max_tokens=100,
        messages=messages
    )
    return {"role": "assistant", "content": response.content[0].text}, {"role": "user", "content": response.content[0].text}

In [115]:


conversation_context_doctor = []
conversation_context_critic = []
doctor_patient_dialogue = conversation_to_string(conversation)
critic_context =  [{"role": "user", "content": "Here is the complete conversation between the Doctor and Patient.\n"+ converted_conversation+ "\n Give feedback to the doctor "}]
critic_feedback = generate_response_critic(critic_context)
conversation_context_critic.append(critic_feedback[0])
critic_feedback_to_doctor = ["Here is some feedback on your previous interaction with the patient\n" + critic_feedback[1]["content"] + "\nThe conversation with the patient will start again. Incorperate the feedback given into your responses"]
conversation_context_doctor.append(
    {"role": "user", "content": critic_feedback_to_doctor},
    {"role": "assistant", "content": "I understand and have acknowleged the feedback. I will incoperate it into the following conversation"}
    {"role": "user", "Now introduce yourself to the patient"}
    )



In [116]:
conversation_context_critic

[{'role': 'assistant',
  'content': "Feedback for the doctor:\n\nYou did a good job of establishing rapport and allowing the patient to share their concerns. However, you could have gathered more detailed information about the patient's symptoms and their impact on their daily life. Additionally, consider exploring the patient's emotional state and providing empathetic responses to build trust."}]

In [117]:
conversation_context_doctor

[{'role': 'user',
  'content': ["Here is some feedback on your previous interaction with the pateintFeedback for the doctor:\n\nYou did a good job of establishing rapport and allowing the patient to share their concerns. However, you could have gathered more detailed information about the patient's symptoms and their impact on their daily life. Additionally, consider exploring the patient's emotional state and providing empathetic responses to build trust.The conversation with the patient will start again. Incoperate the feedback given into your responses"]}]

In [86]:
critic_feedback_to_doctor = ["Here is some feedback on your previous interaction with the pateint" + critic_feedback[1]["content"] + "The conversation with the patient will start again. Incoperate the feedback given into your responses"]
conversation_context_doctor.append({"role": "user", "content": critic_feedback_to_doctor})

In [101]:
conversation_context_critic

In [103]:
print(conversation_context_critic)

None


In [ ]:
critic_feedback_to_doctor = ["Here is some feedback on your previous interaction with the pateint" + critic_feedback[1]["content"] + "The conversation with the patient will start again. Incoperate the feedback given into your responses"]
conversation_context_doctor.append({"role": "user", "content": critic_feedback_to_doctor})
